In [1]:
import pandas as pd
import numpy as np

In [2]:
tweet_data = pd.read_csv(r'C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users_hate_glove.content',header=None, delimiter="\t")
all_data = pd.read_csv(r'C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users_hate_all.content',header=None, delimiter="\t")
# tweet_data = tweet_data.rename(columns={0: "user_id", 301: "hate_label"})

In [3]:
tweet_data.shape,all_data.shape

((100386, 302), (100386, 322))

In [4]:
tweet_data = tweet_data.rename(columns={0: "user_id", 301: "hate_label"})
all_data = all_data.rename(columns={0: "user_id", 321: "hate_label"})

In [5]:
tweet_data = tweet_data[tweet_data['hate_label'] != "other"]

tweet_data.reset_index(inplace=True)
tweet_data.shape

(4971, 303)

In [6]:
all_data = all_data[all_data['hate_label'] != "other"]
all_data.reset_index(inplace=True)
all_data.shape

(4971, 323)

In [7]:

with open(r"C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users.edges", "r") as file:
    lines = file.readlines()
    edges = [tuple(map(int, line.strip().split())) for line in lines]


print("Edges extracted from user.edges file:")
edges[:5]

Edges extracted from user.edges file:


[(10999, 55317),
 (10999, 44622),
 (10999, 10999),
 (10999, 71821),
 (10999, 57907)]

In [8]:
## Removing edges not in our data set
user_id = tweet_data['user_id']
user_id


0            0
1           22
2           29
3           44
4           85
         ...  
4966    100332
4967    100338
4968    100362
4969    100380
4970    100385
Name: user_id, Length: 4971, dtype: int64

In [9]:
valid_nodes = set(user_id)
len(valid_nodes)

4971

In [10]:
cleaned_edges = [(source, target) for source, target in edges if source in valid_nodes and target in valid_nodes]

In [11]:
len(cleaned_edges)

15141

In [12]:
import networkx as nx

# Create an empty graph
tweet_graph = nx.Graph()

# Add nodes from the cleaned dataset
tweet_graph.add_nodes_from(valid_nodes)

# Add edges from the cleaned edges list
tweet_graph.add_edges_from(cleaned_edges)

# You can now perform various operations on the graph
# For example, you can get the number of nodes and edges
num_nodes = tweet_graph.number_of_nodes()
num_edges = tweet_graph.number_of_edges()

# Print the number of nodes and edges
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 4971
Number of edges: 14591


In [13]:
adj_matrix = nx.adjacency_matrix(tweet_graph, nodelist=user_id)
adj_matrix = adj_matrix.toarray()

# Check the number of nodes in the graph
num_nodes = len(user_id)

# Compare the shape of the adjacency matrix with the number of nodes
print("Shape of adjacency matrix:", adj_matrix.shape)
print("Number of nodes in the graph:", num_nodes)

Shape of adjacency matrix: (4971, 4971)
Number of nodes in the graph: 4971


In [14]:

tweet_data.drop(columns=['index'], inplace=True)
all_data.drop(columns=['index'], inplace=True)

In [15]:
tweet_data.head()

,user_id,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,hate_label
0,0,-1.089073,-0.819852,-0.892742,-1.340293,-0.062930,0.328899,0.253635,0.115707,-0.727039,...,-0.185449,-1.034164,-0.923736,0.068635,1.911311,-1.025432,-0.438166,0.582276,0.765071,normal
1,22,-0.020976,-0.658948,-1.183166,-2.388986,0.506060,0.732390,0.298279,-1.706375,-0.149522,...,-1.982040,-1.307111,-1.613784,0.869954,-0.547563,-0.430940,0.233932,1.112722,0.660885,normal
2,29,1.257068,0.523684,-0.423057,-0.642238,-0.072245,-0.471483,-0.338337,0.107110,-1.360391,...,-0.118747,0.323238,-1.461468,-0.324437,0.767427,-0.020313,-0.047015,1.172450,-0.757194,normal
3,44,-0.383711,-0.027758,1.823646,0.944793,0.759426,1.631036,0.775020,0.550644,0.298231,...,-0.872055,0.131165,0.554441,-0.865574,0.555545,0.219754,1.394477,-1.372494,0.165514,normal
4,85,-0.210998,-0.514974,0.184683,0.451372,-0.594404,-0.377794,-0.113593,0.553205,-0.789661,...,-0.039548,-0.004352,0.188056,-0.084436,-0.228187,-0.505434,0.323775,-0.373355,0.058962,normal


In [28]:
tweet_data_features = tweet_data.drop(columns=['hate_label'])
tweet_data_labels = tweet_data['hate_label']

In [29]:
tweet_data_labels

0        normal
1        normal
2        normal
3        normal
4        normal
         ...   
4966     normal
4967     normal
4968     normal
4969    hateful
4970     normal
Name: hate_label, Length: 4971, dtype: object

In [30]:
tweet_data.shape

(4971, 302)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
from torch_geometric.data import Data

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(tweet_data_labels)

edge_index = torch.tensor(list(tweet_graph.edges)).t().contiguous()
data = Data(x=tweet_data_features, edge_index=edge_index)
# Perform train-validation-test split
train_idx, val_idx, test_idx = train_val_test_split(data.num_nodes, val_ratio=0.1, test_ratio=0.1)

# Define train, val, and test masks
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

# Set masks in the data object
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask


# X_train, X_test, y_train, y_test = train_test_split(tweet_data_features, labels_encoded, test_size=0.2, random_state=42)

# # Assuming 'tweet_graph' is your networkx graph
# # You can convert it to PyTorch Geometric format if necessary

# # Convert the graph to PyTorch Geometric format if needed
# # Assuming 'tweet_graph' is an adjacency matrix

# tweet_graph_tensor = torch.tensor(adj_matrix, dtype=torch.long).t().contiguous()
# train_graph,test_graph = train_test_split(tweet_graph_tensor, test_size=0.2, random_state=42)
# train_graph.shape


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import networkx as nx
import torch
from torch_geometric.data import Data

# Assume you have a graph G and node features x

# Convert the NetworkX graph to a PyTorch Geometric Data object
data = Data(x=torch.tensor(x), edge_index=torch.tensor(list(train_graph.edges)).t().contiguous())

# Perform train-validation-test split
train_idx, val_idx, test_idx = train_val_test_split(data.num_nodes, val_ratio=0.1, test_ratio=0.1)

# Define train, val, and test masks
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

# Set masks in the data object
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [20]:
y_train

array([1, 1, 1, ..., 1, 1, 0])

In [21]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = torch.mean(x, dim=0)  # Global pooling
        x = torch.sigmoid(self.fc(x))
        return x

In [22]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [23]:
# Initialize the model
input_dim = X_train_tensor.shape[1]
hidden_dim = 64
output_dim = 1
model = GCN(input_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [24]:
# Training the model
test_graph = test_graph.t()
train_graph = train_graph.t()
model.train()
for epoch in range(10):
    for data in train_loader:
        optimizer.zero_grad()
        print("Input data shape:", data[0].shape)
        print("Graph data shape:", train_graph.shape)
        outputs = model(data[0], train_graph)  # Pass features and graph data
        loss = criterion(outputs, data[1].view(-1, 1))
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


Input data shape: torch.Size([64, 300])
Graph data shape: torch.Size([4971, 3976])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 4971 but got size 2 for tensor number 1 in the list.